In [1]:
modelname = "40_unnormalised/checkpoint-6700/"

from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC

processor = Wav2Vec2Processor.from_pretrained(modelname)
model = Wav2Vec2ForCTC.from_pretrained(modelname)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
vocab_dict = processor.tokenizer.get_vocab()
sorted_vocab_dict = {k: v for k, v in sorted(vocab_dict.items(), key=lambda item: item[1]) if k != "<pad>"}

In [3]:
from pyctcdecode import build_ctcdecoder

decoder = build_ctcdecoder(
    labels=list(sorted_vocab_dict.keys()),
    kenlm_model_path="5gram.bin",
)

Unigrams not provided and cannot be automatically determined from LM file (only arpa format). Decoding accuracy might be reduced.
Found entries of length > 1 in alphabet. This is unusual unless style is BPE, but the alphabet was not recognized as BPE type. Is this correct?
No known unigrams provided, decoding results might be a lot worse.


In [4]:
from transformers import Wav2Vec2ProcessorWithLM

processor = Wav2Vec2ProcessorWithLM(
    feature_extractor=processor.feature_extractor,
    tokenizer=processor.tokenizer,
    decoder=decoder,
)

In [12]:
processor.save_pretrained("40_with_lm/wav2vec2-xls-r-parlaspeech-hr",
)

In [13]:
model.save_pretrained("40_with_lm/wav2vec2-xls-r-parlaspeech-hr",)

In [5]:

import pandas as pd
import numpy as np
import os

transferdir = "transfer/"

def process(text: str):
    from parse import compile
    from string import punctuation

    p = compile("{hit:d}.")
    in_list = text.split()
    out_list = list()
    for seg in in_list:
        parse_result = p.parse(seg)
        if parse_result:
            # We got a number with a dot afterward:
            out_list.append(seg.lower())
        else:
            out_list.append(seg.translate(str.maketrans("", "", punctuation)).lower())
    return " ".join(out_list)

df = pd.read_json("ParlaSpeech-HR.v1.0.jsonl", orient="records", lines=True)
df = df.rename(columns={"path":"hashname"})
df = df.loc[df.split.isin("test,dev".split(",")), :]

df["sentence"] = df.words.apply(" ".join).apply(process)
df["path"] = df.hashname.apply(lambda s: os.path.join(transferdir, s))


def get_transcript(audio_filepath:str):
    import soundfile as sf
    import torch

    speech, sample_rate = sf.read(audio_filepath)
    input_values = processor(speech, sampling_rate=sample_rate, return_tensors="pt").input_values.cuda()
    inputs = processor(speech, sampling_rate=sample_rate, return_tensors="pt")
    with torch.no_grad():
      logits = model(**inputs).logits
    transcription = processor.batch_decode(logits.numpy()).text

    return transcription[0]

from datasets import load_metric
wer_metric = load_metric("wer")
cer_metric = load_metric("cer")
from tqdm.auto import tqdm
df["predictions"] = [get_transcript(path) for path in tqdm(df.path.values)]
for splt in ["dev", "test"]:
    print("Evaluating on ", splt)
    wer = wer_metric.compute(
        references=df.loc[df.split==splt, "sentence"],
        predictions=df.loc[df.split==splt, "predictions"]
    )

    cer = cer_metric.compute(
        references=df.loc[df.split==splt, "sentence"],
        predictions=df.loc[df.split==splt, "predictions"]
    )

    print(f"{wer=:0.4f}, {cer=:0.4f}")

  0%|          | 0/1013 [00:00<?, ?it/s]

Evaluating on  dev
wer=0.0556, cer=0.0253
Evaluating on  test
wer=0.0430, cer=0.0188


In [6]:
df[["sentence", "predictions", "split", "hashname"]].to_csv(
    "41_model_with_LM_output_for_evaluation.csv", 
    index=False
    )

In [7]:
pd.set_option('display.max_colwidth', None)
df["wer"] = df.apply(
    lambda row: wer_metric.compute(
        references = [row["sentence"]],
        predictions= [row["predictions"]]),
        axis=1
)

df["cer"] = df.apply(
    lambda row: cer_metric.compute(
        references = [row["sentence"]],
        predictions= [row["predictions"]]),
        axis=1
)
df.sort_values(by="cer", ascending=False)[["sentence", "predictions", "cer"]].head(10)

,sentence,predictions,cer
1545,iznio stav kluba hrvatske demokratske zajednice to ću kad budem imao prigode reći što mislim reći o,iznio stav kluba hdza to ću kad bude imao prigode reći što mislim o,0.323232
259786,predsjednika dr franje tuđmana,predsjednika doktora franjeto tuđmana,0.233333
70625,predsjednika republike f tuđmana,predsjednika republike franje tuđmana,0.156250
2094,subvencioniranju stambenih kredita je osmišljen 2011g doduše tada je uz subvenciju išla i pratilo ju je državno jamstvo i,subvencionranju stambenih kredita je osmišljen 2011. godine do duše tada je uz njega uz subvenciju išla i pratilo ju je državno jamstvo i,0.148760
121872,evo ja bih samo htio napomenuti da,ja bi samo htio napomenuti da,0.147059
2200,krava je li vi znate da mi danas da ne uvozimo hranu mi bi bili glasni mi bi bili gladni u hrvatskoj koja ima ¾ plodne zemlje savršenu klimu koja je nekada hranila 5 država mi bi gladovali,krava jer vi znate da mi danas da ne uvozimo hranu mi bi bili gladni mi bi bili gladni u rh koja ima 34vtineplodne zemlje savršenu klimu ne znam koja je nekada hranila 5 država mi bi gladovali,0.143617
121858,tzv profesionalnim žaliteljima kao što sam rekla a to smo u jednom dijelu i referirali se na to izvješće za 2018 ukazali smo znači da postoji,tzv profesionalnim žaliteljima kao što sam rekla to smo u jednom dijelu i referirali se na to u izvješću za 2018. godinu ukazali smo da postoje,0.141844
196052,primjer eu odnosno istaknuto je da 18 država članica bilježilo je porast broja stanovnika međutim povećan porast broja stanovnika zabilježen je prije svega zbog migracija u europu je 18g doselilo nekih 3 milijuna i 225 tisuća stanovnika ali istovremeno je od,primjer europske unije odnosno istaknuto je da 18 država članica bilježilo je porast broja stanovnika međutim uz porast broja stanovnika zabilježen je prije svega zbog migracija u europu 18 godine doselilo nekih 3 milijuna i 225 000 stanovnika ali istovremeno je,0.139535
1354,kada se samo sjetimo onih tužnih brojki od prije tri četiri godine oni su za ove brojke sada dobre na žalost bile su ružne ali šta se sada dešava,kada se samo sjetimo onih tužnih brojki od prije 3 4 godine oni su za ove brojke sada izgledaju dobro na žalost bile su ružne ali šta se sada dešava,0.137931
1851,ponekad banke naplaćuju promjenu,ponekad banke nam naplaćuju promjenu,0.125000
